In [1]:
import numpy as np
import pandas as pd
import scipy as sp

import matplotlib.pyplot as plt

import os
import glob

from nfl import Team, Game, Play, Player

In [2]:
games = pd.read_csv('data/games.csv')

In [3]:
players = pd.read_csv('data/players.csv').set_index('nflId')

In [4]:
plays = pd.read_csv('data/plays.csv')

In [5]:
weeks = {}
for i in range(1):
    name = f'week{i+1}'
    weeks[name] = pd.read_csv(os.path.join('data',name+'-processed.csv'))

In [6]:
team_names = games['homeTeamAbbr'].unique()

In [7]:
teams = {}
for name in team_names:
    teams[name] = Team(abbr=name)

In [8]:
for i in range(games.shape[0]):
    _game_info = games.loc[i]
    gameId = _game_info['gameId']
    homeTeamAbbr = _game_info['homeTeamAbbr']
    visitorTeamAbbr = _game_info['visitorTeamAbbr']
    week = _game_info['week']
    
    game_plays = plays[plays['gameId'] == gameId]
    home_game_plays = game_plays[game_plays['possessionTeam']==homeTeamAbbr].sort_values(by=['playId']).reset_index(drop=True)
    away_game_plays = game_plays[game_plays['possessionTeam']==visitorTeamAbbr].sort_values(by=['playId']).reset_index(drop=True)
    
    teams[homeTeamAbbr].games[f'week{week}'] = Game(gameId,opponent=visitorTeamAbbr,game_info=_game_info,
                                                    play_data=away_game_plays,location='home')
    teams[visitorTeamAbbr].games[f'week{week}'] = Game(gameId,opponent=homeTeamAbbr,game_info=_game_info,
                                                    play_data=home_game_plays,location='away')

In [9]:
team = teams['NYG']

In [10]:
for key in ('week1',):
    game = team.games[key]
    
    week_data = weeks[key]
    
    for i in game.play_data.index:
        play = game.play_data.loc[i]
        tracking_data = week_data[(week_data['gameId']==play['gameId'])&(week_data['playId']==play['playId'])]
        player_tracking = tracking_data[tracking_data['nflId'].notna()]
        fb_tracking = tracking_data[tracking_data['displayName']=='Football'].sort_values(by='frameId').reset_index(drop=True)
        
        game.plays.append(Play(play['playId'],play_data=play,player_tracking=player_tracking,
                               fb_tracking=fb_tracking,defensive_team=game.location))

In [11]:
game = team.games['week1']

In [12]:
game.opponent

'JAX'

In [13]:
game.location

'home'

In [14]:
game.nPlays

34

In [15]:
game.list_plays()

Play 1: (14:19) (Shotgun) B.Bortles pass deep left to K.Cole to 50 for 31 yards (J.Jenkins).
Play 2: (11:42) (Shotgun) B.Bortles pass incomplete short right to D.Westbrook.
Play 3: (11:36) (Shotgun) B.Bortles pass short right to D.Westbrook to NYG 23 for 6 yards (E.Apple) [C.Barwin]. Penalty on NYG-R.Armstrong, Defensive Offside, declined.
Play 4: (10:27) (Shotgun) B.Bortles pass incomplete short left to T.Yeldon.
Play 5: (10:22) (Shotgun) B.Bortles pass incomplete deep right to D.Moncrief (E.Apple).
Play 6: (7:04) B.Bortles pass short right to L.Fournette ran ob at JAX 46 for 11 yards (A.Ogletree).
Play 7: (6:31) (Shotgun) B.Bortles sacked at JAX 38 for -8 yards (sack split by K.Wynn and B.Goodson).
Play 8: (5:47) (Shotgun) B.Bortles pass deep left intended for D.Moncrief INTERCEPTED by J.Jenkins at NYG 37. J.Jenkins ran ob at NYG 37 for no gain.
Play 9: (13:02) (Shotgun) B.Bortles pass short middle to T.Yeldon to NYG 49 for 7 yards (K.Wynn, R.Armstrong).
Play 10: (12:32) (Shotgun) B.